In [ ]:
%load_ext autoreload
%autoreload 2

# Run conversion script for Raw_Intan_Ephys_withVideos

In [ ]:
from nelson_lab_to_nwb.intan_sessions_converter import IntanSessionNWBConverter
from neuroconv.utils.dict import load_dict_from_file, dict_deep_update
from pynwb import NWBHDF5IO, NWBFile

In [ ]:
# set the source data for the Converter
source_data = dict(
    IntanMultifilesRaw=dict(
        folder_path="/media/luiz/Seagate Basic/storage/catalystneuro/nelson_lab/Raw_Intan_Ephys_withVideos/pm231213a_240205_240205_120954"
    ),
    AIMScore=dict(
        file_path="/media/luiz/Seagate Basic/storage/catalystneuro/nelson_lab/Raw_Intan_Ephys_withVideos/AIM_score.xlsx",
    )
)

# Instantiate the Converter
converter = IntanSessionNWBConverter(source_data)

# Automatically fetch metadata from files, then update it with user-defined metadata
source_metadata = converter.get_metadata()
user_metadata_file = "/mnt/shared_storage/Github/nelson-lab-to-nwb/src/nelson_lab_to_nwb/intan_sessions_converter/metadata_example.yaml"
user_metadata = load_dict_from_file(file_path=user_metadata_file)
metadata = dict_deep_update(source_metadata, user_metadata)

In [ ]:
# Conversion options
conversion_options = dict(
    IntanMultifilesRaw=dict(
        stub_test=True,
    )
    AIMScore=dict(
        timestamp_column_name="Time (minutes relative to injection)",
        aims_column_name="AIMS",
    )
)

# Run conversion
converter.run_conversion(
    metadata=metadata,
    nwbfile_path="converted_intan_session.nwb",
    overwrite=True,
    conversion_options=conversion_options
)

In [ ]:
io = NWBHDF5IO("converted_intan_session.nwb", 'r', load_namespaces=True)
nwbfile = io.read()
nwbfile

# Multiple Intan files

ref: https://github.com/SpikeInterface/spikeinterface/issues/2620

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from nelson_lab_to_nwb.interfaces.intan_multi_interface import IntanMultifilesRecordingInterface
from datetime import datetime
from dateutil import tz
from pynwb import NWBHDF5IO, NWBFile

In [ ]:
folder_path = "/media/luiz/Seagate Basic/storage/catalystneuro/nelson_lab/Raw_Intan_Ephys_withVideos/pm231213a_240205_240205_120954"
interface = IntanMultifilesRecordingInterface(folder_path=folder_path, verbose=False)

# Extract what metadata we can from the source files
metadata = interface.get_metadata()
# session_start_time is required for conversion. If it cannot be inferred
# automatically from the source files you must supply one.
session_start_time = datetime(2020, 1, 1, 12, 30, 0, tzinfo=tz.gettz("US/Pacific"))
metadata["NWBFile"].update(session_start_time=session_start_time)

In [ ]:
nwbfile_path = "intan_data.nwb"
interface.run_conversion(
    nwbfile_path=nwbfile_path,
    metadata=metadata,
    overwrite=True
)

In [ ]:
io = NWBHDF5IO("intan_data.nwb", 'r', load_namespaces=True)
nwbfile = io.read()
nwbfile

# AIM Score

In [ ]:
from nelson_lab_to_nwb.interfaces import AIMScoreInterface

In [ ]:
file_path = "/media/luiz/Seagate Basic/storage/catalystneuro/nelson_lab/Raw_Intan_Ephys_withVideos/AIM_score.xlsx"
interface = AIMScoreInterface(
    file_path=file_path,
    # timestamp_column_name="Time (minutes relative to injection)",
    # aims_column_name="AIMS",
    verbose=True
)